In [ ]:
import requests
import pandas as pd
import datetime

import json
import os.path
import glob

In [ ]:
today = pd.Timestamp.now().strftime('%Y_%m_%d_%H_%M_%S')

# Get Player IDs

In [ ]:
ref_list = []
for num in range(1, 7):
    url = 'https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes?limit=1000&active=true&page={}'.format(num)

    r = requests.get(url)
    d = r.json()

    ref_list += d['items']

file = 'links_for_player_ids' + '_' + today + '.json'
with open(file, 'w') as f:
    json.dump(ref_list, f, ensure_ascii=False, indent=4)

In [ ]:
list_of_files = glob.glob('links_for_player_ids*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

In [ ]:
player_id_data = json.load(open(latest_file))

# Get Specific Player Info/QBRs for NFL

In [ ]:
player_ids = []
for player in range(0, len(player_id_data)):
    player_ids.append(player_id_data[player]['$ref'][72:int(player_id_data[player]['$ref'].find('?'))])

In [ ]:
full_dict = {}

for player_id in player_ids:

    # This gets every single players info (height, weight, team, etc.)
    info_url = "http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes/{}?lang=en&region=us".format(player_id)
    
    player_info_req = requests.get(info_url)
    player_info = player_info_req.json()

    # Filters for QBs only
    if player_info['position']['id'] == '8':
        # This gets all the stats for every season a QB played in the NFL
        # Note: If no stats this pops an error
        seasons_url = 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/athletes/{}/statisticslog?lang=en&region=us'.format(player_id)

        player_seasons_req = requests.get(seasons_url)
        player_seasons = player_seasons_req.json()

        # Iterates through every link to each season of stats
        if 'entries' in player_seasons.keys():
            print(player_id, player_info['displayName'])
            if 'country' in player_info['birthPlace'].keys():
                country = player_info['birthPlace']['country']
            else:
                country = 'Not Given'
                
            full_dict[player_id] = {'player_name':player_info['displayName'],
                                    'date_of_birth':player_info['dateOfBirth'],
                                    'city':player_info['birthPlace']['city'],
                                    'state':player_info['birthPlace']['state'],
                                    'country':country,
                                    'height':player_info['height'],
                                    'weight':player_info['weight']
                                   }

            # This is done in case the player wasn't drafted or is 
            # a rookie. If either are true, this pops an error
            if 'debutYear' in player_info.keys():
                full_dict[player_id]['debut_year'] = player_info['debutYear']
            elif 'debutYear' not in player_info.keys() and player_info['experience']['years'] == 0:
                full_dict[player_id]['debut_year'] = 2023
            elif 'debutYear' not in player_info.keys() and player_info['experience']['years'] != 0:
                full_dict[player_id]['debut_year'] = player_seasons['entries'][-1]['season']['$ref'][71:75]
            else:
                raise Exception("Player is not an undrafted free agent or in debut year") 
                
            
            # This is done for Undrafted Free Agents. It pops an error if
            # they weren't drafted
            if 'draft' in player_info.keys():
                full_dict[player_id]['draft'] = {
                                        'year':player_info['draft']['year'],
                                        'round':player_info['draft']['round'],
                                        'selection':player_info['draft']['selection']
                                    }
            else:
                full_dict[player_id]['draft'] = 'UDFA'

            for stats in player_seasons['entries']:
                nfl_stats = {}
                player_stats_req = requests.get(stats['statistics'][0]['statistics']['$ref'])
                player_stats = player_stats_req.json()
    
                # Gets all types of NFL QBRs for each season
                for qbr in player_stats['splits']['categories'][1]['stats']:
                    if ("QBR" in qbr['name']) or ("quarterbackRating" in qbr['name']):
                        year = stats['statistics'][0]['statistics']['$ref'][71:75]
                        if year in nfl_stats.keys():
                            nfl_stats[year].update({qbr['name']:qbr['value']})
                        else:
                            nfl_stats[year] = {qbr['name']:qbr['value']}
                            
                if 'nfl' not in full_dict[player_id].keys():
                    full_dict[player_id]['nfl'] = nfl_stats
                else:
                    full_dict[player_id]['nfl'].update(nfl_stats)

        # College stats start here
        if 'collegeAthlete' in player_info.keys():
            college_stats_req = requests.get(player_info['collegeAthlete']['$ref'])
            college_stats = college_stats_req.json()
            
            if 'statisticslog' in college_stats.keys():
                stat_by_qb_req = requests.get(college_stats['statisticslog']['$ref'])
                stat_by_qb = stat_by_qb_req.json()

                for college_year in stat_by_qb['entries']:
                    college_stats = {}
                    for stat_type in college_year['statistics']:
                        if stat_type['type'] == 'total':
                            year = stat_type['statistics']['$ref'][84:88]

                            actual_stats_req = requests.get(stat_type['statistics']['$ref'])
                            actual_stats = actual_stats_req.json()
                            
                            for stat in actual_stats['splits']['categories']:
                                for value in stat['stats']:
                                    if (stat['name'] == 'general' and value['name'] == 'gamesPlayed') or \
                                        (stat['name'] in ['passing', 'rushing', 'scoring']):
                                        if year in college_stats.keys():
                                            college_stats[year].update({value['name']:value['value']})
                                        else:
                                            college_stats[year] = {value['name']:value['value']}

                            if player_id not in full_dict.keys():
                                pass
                            else:
                                if 'college' not in full_dict[player_id].keys():
                                    full_dict[player_id]['college'] = college_stats
                                else:
                                    full_dict[player_id]['college'].update(college_stats)
                    
            else:
                full_dict[player_id]['college'] = None
        else:
            full_dict[player_id]['college'] = None

    else:
        pass

In [ ]:
file = 'player_stats_dictionary' + '_' + today + '.json'
with open(file, 'w') as f:
    json.dump(full_dict, f, ensure_ascii=False, indent=4)